<img src="https://github.com/simonwiles/colab_workshops/raw/master/cidr-logo.no-text.240x140.png" style="display:block;margin:0 auto;" alt="Center for Interdisciplinary Digital Research @ Stanford"/>
<H1 align="center">Digital Tools and Methods for the Humanities and Social Sciences</H1>
<H2 align="center">Data Manipulation with Python</H2>

### Instructors
- Scott Bailey (CIDR), <em>scottbailey@stanford.edu</em>
- Simon Wiles (CIDR), <em>simon.wiles@stanford.edu</em>

### Goal
By the end of this workshop, we hope you'll be able to load in data into a Pandas `DataFrame`, perform basic cleaning and analysis, and create visualizations of some relevant aspects of a dataset.  For most of this workshop we will work with a dataset prepared from the [IMDb Datasets](https://www.imdb.com/interfaces/) and the [OMDb API](https://www.omdbapi.com/).

### Topics
- Pandas Series and DataFrame
- Loading data in, null and missing data
- Describing data
- Column manipulation
- String manipulation
- Split-Apply-Combine
- Plotting:
  - Basic charts (line, bar, pie)
  - Histograms
  - Scatter plots
  - Boxplots, violinplots
  
<mark>TODO: the above should probably be reworked</mark>

### Jupyter Notebooks and Google Colaboratory

Jupyter notebooks are a way to write and run Python code in an interactive way. They're quickly becoming a standard way of putting together data, code, and written explanations or visualizations into a single document and sharing that. There are a lot of ways that you can run Jupyter notebooks, including just locally on your computer, but we've decided to use Google's Colaboratory notebook platform for this workshop. Colaboratory is a cloud-based platform that allows you ~to create libraries, which are effectively project folders and virtual environments that can contain static files and Python notebooks. They come with a number of popular libraries pre-installed, and allow you to install other libraries as needed.~

Using the Google Colaboratory platform allows us to focus on learning and writing Python in the workshop rather than on setting up Python, which can sometimes take a bit of extra work depending on platforms, operating systems, and other installed applications. If you'd like to install a Python distribution locally, though, we have some instructions (with gifs!) on installing Python through the Anaconda distribution, which will also help you handle virtual environments: https://github.com/sul-cidr/python_workshops/blob/master/setup.ipynb <mark> ← TODO: migrate this to a wiki page on the CIDR Workshops repo</mark>

If you run into problems, or would like to look into other ways of installing Python or handling virtual environments, feel free to send us an email (contact-cidr@stanford.edu) or visit us during our [consulting hours](https://library.stanford.edu/research/cidr/consulting).

~For now, go ahead to https://notebooks.azure.com and login with your Stanford ID and password.~

### Environment
If you would prefer to use Anaconda or their own local installation of python or Jupyter Notebooks, for this workshop you will need an environment with the following packages installed and available:
- `pandas`
- `matplotlib`
- `requests`
- `sqlalchemy`
- `seaborn` (available in the `conda-forge` channel)

Please note that we will likely not have time during the workshop to support you with problems related to a local environment, and we do recommend using the Colaboratory notebooks if you are at all unsure.

###  Copying this notebook
~Go to https://notebooks.azure.com/versae/libraries/cidr-data-manipulation~
    
~From there, click "Clone" to create a full copy of this library.~

## 1. What is Pandas?

Pandas is a high-level data manipulation tool first created in 2008 by Wes McKinney.  The name is derived from the term “panel data,” an econometrics term for data sets that include observations over multiple time periods for the same individuals.<sup>[[wikipedia](https://en.wikipedia.org/wiki/Pandas_(software))]</sup>

From Jake Vanderplas’ book [**Python Data Science Handbook**](http://shop.oreilly.com/product/0636920034919.do) (from which some code excerpts are used in this workshop):

> Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a `DataFrame`. `DataFrame`s are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

In [ ]:
import pandas as pd

# The two lines below configure how our outputs are shown in this notebook
#  environment.  They need not concern us now.
pd.set_option('display.max_rows', 20)
pd.DataFrame._repr_html_ = \
    lambda self: ('<style>table.dataframe td {white-space: nowrap}</style>' +
                  self.to_html(max_rows=10, show_dimensions=True, notebook=True))

### 1.1. What does Pandas *do*?

<mark>TODO: wip</mark>
* Reading and writing data from persistent storage
* Cleaning, filtering, and otherwise preparing data
* Calculating statistics and analyzing data


... but perhaps we should let Pandas introduce itself:

In [ ]:
pd?

### 1.2. Where can I get more help with Pandas?

The [Pandas website](https://pandas.pydata.org/) and [online documentation](http://pandas.pydata.org/pandas-docs/stable/) are useful resources, and of course the indispensible [Stack Overflow](https://stackoverflow.com/questions/tagged/pandas) has a "pandas" tag, and there is also a (much younger, much smaller) sister [site dedicated to Data Science questions](https://datascience.stackexchange.com/questions/tagged/pandas) that has a "pandas" tag too.

In [ ]:
pd.isnull?

## 2. Introduction to `DataFrame`s and `Series`

The main data structure that Pandas implements is the `DataFrame`, and a `DataFrame` is composed of one or more `Series` and, optionaly, an `Index`.  

A `DataFrame` is a two-dimensional array with flexible row indices and flexible column names. It can be thought of as a generalization of a two-dimensional NumPy array, or a specialization of a dictionary in which each column name maps to a `Series` of column data.

A `Series` is a one-dimensional array of indexed data. It can be thought of as a specialized dictionary or a generalized NumPy array.

A `DataFrame` is made up of `Series` in a similar way in which a table is made up of columns. The only restriction is that each column must be of the same data type.  Many of the operations that can be performed on a `DataFrame` can also be performed on an individual `Series`.


<mark>**GRAPHIC HERE**</mark>

## 3. Creating `DataFrame`s and loading data

There are a great many ways to create a Pandas `DataFrame` -- we can build one ourselves in lower-level Python datatypes, of course, but Pandas also provides methods to load data in from common storage and serialization formats.

<a title="PerryPlanet [Public domain], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Bayarea_map.svg" style="float:right"><img width="256" alt="Bayarea map" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Bayarea_map.svg/512px-Bayarea_map.svg.png"></a>
### 3.1. Introduction to `DataFrame`s

The simplest way to generate a `DataFrame` is to create it directly from a `dict` of `list`s:

In [ ]:
data = {
    "county": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"],
    "county seat": ["Oakland", "Martinez", "San Rafael", "Napa", "San Francisco", "Redwood City", "San Jose", "Fairfield", "Santa Rosa"],
    "population": [1494876, 1037817, 250666, 135377, 870887, 711622, 1762754, 411620, 478551],
    "area": [2130, 2080, 2140, 2040, 600.59, 1930, 3380, 2350, 4580]
}
bay_area_counties = pd.DataFrame(data)
bay_area_counties

Pandas has automatically created an `Index` on this `DataFrame` ([0..8]), but we can also specify our own `Index` when we instantiate the frame ourselves:

In [ ]:
bay_area_counties = pd.DataFrame(data, index=["Ala", "Con", "Mar", "Nan", "SF", "SM", "SC", "Sol", "Son"])
bay_area_counties

This allows us to `loc`ate a specific reference using the key in the `Index`:

In [ ]:
bay_area_counties.loc['Ala']

We can also set an `Index` at any time after the `DataFrame` has been created, either by adding a new index:

In [ ]:
bay_area_counties = pd.DataFrame(data)
bay_area_counties.index = ["Ala", "Con", "Mar", "Nan", "SF", "SM", "SC", "Sol", "Son"]
bay_area_counties

or by choosing one of the existing columns to become the index: <mark>(note the use of `inplace=True`)</mark>

In [ ]:
bay_area_counties = pd.DataFrame(data)
bay_area_counties.set_index('county', inplace=True)
bay_area_counties

In [ ]:
bay_area_counties.loc['Santa Clara']

### 3.2. Reading data from persistent storage

However, most of the time we're more likely to be reading data in from an external source of some kind, and Pandas has us well covered here.

First, let's grab some data into our Colaboratory Notebook environment so that we can work with it locally:

In [ ]:
!mkdir -p workshop_data
!wget https://raw.githubusercontent.com/simonwiles/colab_workshops/master/sample_data/imdb_top_1000.csv -O workshop_data/imdb_top_1000.csv

#### 3.2.1. CSV files
Reading in data from CSV files is as simple as:

In [ ]:
data_frame = pd.read_csv('workshop_data/imdb_top_1000.csv')
data_frame

Notice again that Pandas has created a default `Index` for this `DataFrame` -- we probably want the `imdbID` column to be the `Index`, and we can set that after import, as above, or we can specify it when loading the CSV initially:

In [ ]:
data_frame = pd.read_csv('workshop_data/imdb_top_1000.csv', index_col='imdbID')
data_frame.head(4) # the `head` method defaults to five if called without an argument
                   # a `tail` method is also available with the same semantics

#### 3.2.2. Reading data from JSON Files

JSON files can be loaded in a similarly straightforward way.

There are two things to note here:

1. the nature of JSON as a file format is such that the `Index` is explicit, and Pandas will set it correctly for us initially.
2. We're loading the data directly over HTTP(S) here -- Pandas `read_...` methods can accept a local file path or a URL, and Pandas will take care of fetching the data for you.

In [ ]:
pd.read_json('https://raw.githubusercontent.com/simonwiles/colab_workshops/master/sample_data/imdb_top_1000.json')

#### 3.2.3. Reading data via a SQL query

We can also load data from relational databases or other datastores that export a SQL-compatible interface.  For this example we'll download a simple SQLite database file to operate on, but Pandas’ `read_sql*` methods can accept a `connection` object that is predicated on a remote database server if required.

In [ ]:
!pip install sqlalchemy
!mkdir -p workshop_data
!wget https://raw.githubusercontent.com/simonwiles/colab_workshops/master/sample_data/imdb_top_1000.sqlite -O workshop_data/imdb_top_1000.sqlite

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///workshop_data/imdb_top_1000.sqlite", echo=False)
pd.read_sql_query("SELECT * FROM imdb_top_1000;", con=engine, index_col='imdbID')

#### 3.2.3. Other input formats

Pandas also has methods that allow it to read data directly from other formats, including those used by Microsoft Excel, Stata, SAS, and Google Big Query.  More details are available from the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

### 3.3. Writing `DataFrames` back out to persistent storage

Pandas makes writing data to persistent storage formats similarly convenient.  Methods are available to write to most of the formats Pandas can read, including all those demonstrated above.

In [ ]:
data_frame.to_csv('imdb_data_2.csv')

## 4. Working with `DataFrame`s

Let's begin by loading our dataset of the top 1,000 ranked films on imDB.

<mark>In the same way that it's common to `import pandas as pd`, it's common to use `df` as an identifier for generic `DataFrame`s, especially in tutorials and demos.  For anything other than interactive sessions or throw-away scripts, I strongly recommend using good descriptive identigiers for your `DataFrame`s!</mark>

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/simonwiles/colab_workshops/master/sample_data/imdb_top_1000.example.csv', index_col='imdbID')

### 4.1. Exploring `DataFrame`s

Pandas provides a host of ways to explore our data, which is especially useful when we're getting to know a new dataset or source.

In [ ]:
df.info()

In [ ]:
df.Rated.unique()

In [ ]:
df.Rated.value_counts()

In [ ]:
df.imdbVotes.describe()

### 4.2. Indexing and slicing
Accessing columns can be done using the dot notation, `df.column_name`, or the dictionary notation, `df['column_name']`.  This returns a `Series` object.

In [ ]:
df.Title

In [ ]:
type(df.Title)

`DataFrame`s can be sliced to return a subset of the available columns -- this returns a new `DataFrame` object.

In [ ]:
df[['Title', 'Director']]

In [ ]:
type(df[['Title', 'Director']])

`Series` object and other subsets of `DataFrame`s preserve the indices of the `DataFrame` from which they are derived, which makes further operations such as merging or columns manipulation possible.

`DataFrame`s are designed to operate at the column level, not at the row level. However, a subset of rows can be visualized easily using a slice like in any Python list.

In [ ]:
df[10:15]

We can chain these operations together, as long as we remember what we are returning in each link of the chain.

In [ ]:
df[10:15].Production

In [ ]:
df.Production[10:15]

In [ ]:
df[['Production']][10:15]

In [ ]:
df[["Title", "Plot"]].iloc[2:5]

In [ ]:
df[["Title", "Plot"]].loc['tt7286456':'tt5813916']

<mark>Notice how in then `.iloc` example, record #5 is omitted, in usual python slice fashion, but in the `.loc` example, the observation with `imdbID == tt5813916` is *included* in the result.</mark>

#### 4.2.1 Activity

Given the `DataFrame` defined above, write an expression to extract a `DataFrame` with the columns `Title`, `Year`, `Director`, and `imdbRating`. Show only the first 5 rows of it.

In [ ]:
# Write your code here

### 4.3. Expressions
<mark>TODO: intro.</mark>

Operations performed on a column, or `Series`, are broadcast to each of the elements.

In [ ]:
# convert the running times to hours
df.Runtime / 60

Simple string concatenation can be performed in the same manner:

In [ ]:
df.Title + '(' + df.Rated + '), directed by: ' + df.Director

as are boolean operations:

In [ ]:
df.Year < 2000

By itself this is not terribly useful, but expressions of this kind are very powerful when passed to a `DataFrame` to select content:

In [ ]:
df[df.Production == 'Paramount Pictures']

In [ ]:
df[df.Year < 2000]

Any expression that evaluates to a `Series` of boolean values (`True` or `False`) and shares the same `Index` as the source `DataFrame` can be used.  Complex conditions can be assembled using bitwise logical operators `&`, `|`, and `~` to create simple but powerful filters.

In [ ]:
# returns a `Series`
df[(df.Year < 2000) & (df.imdbRating > 8)].Title

In [ ]:
# returns a `DataFrame`
df.loc[(df.Year < 2000) & (df.imdbRating > 8), ['Title', 'Year', 'imdbRating']]

The `.str` property gives access to string variables in a broadcast fashion, such that they can be manipulated:

In [ ]:
df.Actors.str.split(', ')

We can make use of `.str` in expressions:

In [ ]:
# select records with Oscar nominations or wins
df[df.Awards.str.contains('Oscar') == True]

In [ ]:
df[df.Director.apply(lambda a: len(a.split(', ')) > 3)]

#### 4.3.1. Activity

Returning to our “Bay Area Counties” data from earlier, write an expression to calculate the population density of each county.

In [ ]:
data = {
    "county": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"],
    "county seat": ["Oakland", "Martinez", "San Rafael", "Napa", "San Francisco", "Redwood City", "San Jose", "Fairfield", "Santa Rosa"],
    "population": [1494876, 1037817, 250666, 135377, 870887, 711622, 1762754, 411620, 478551],
    "area": [2130, 2080, 2140, 2040, 600.59, 1930, 3380, 2350, 4580]  # data is in km²
}
bay_area_counties = pd.DataFrame(data)
with pd.option_context('display.max_rows', 10):
    display(bay_area_counties)

In [ ]:
# Write your code here

### 4.4. Cleaning and manipulating `DataFrame`s

<mark>TODO: something on basic assignment to `DataFrame`s?</mark>

The fundamental way of manipulating the contents of `DataFrame`s is by using the `apply()` method.  `apply()` takes a function as an argument, and `apply`s the function to each element in the container it’s called on.

In [ ]:
def count_values_naive(text):
    values = text.split(", ")
    return len(values)

In [ ]:
df.Genre

In [ ]:
df.Genre.apply(count_genres_naive)

Unfortunately our `count_genres_naive` function does not know how to handle missing data.  (Pandas uses the `NaN` datatype from the underlying `numpy` packages to represent missing data, and this datatype is based on the primitive `float` type, which is why the `Attribute` error reads as it does.  Note that if the missing data was represented as `None`, the same problem would arise, and if it was represented by the empty string `''` it would be worse, as `len(''.split(',')) == 1`!)

We could handle this problem in a number of ways:
1. we could modify our `count_genres_naive` function to handle the missing values;
2. we could drop the observations with missing values immediately (and temporarily) before we apply our count function; or
3. we could clean the dataset when we initially import it.

In [ ]:
df.Genre.dropna()

In [ ]:
df.Genre.dropna().apply(count_genres_naive)

In [ ]:
df['genre_count'] = df.Genre.dropna().apply(count_genres_naive)
df[pd.isna(df.Genre)][['Genre', 'genre_count']]

One problem with this approach is that we 

In [ ]:
df.isnull().sum()

### 4.4.1 Cleaning on import


### 4.4.2 Activity

Add a new column `language_count` to the `DataFrame` and show movies with more than 1 language.

In [ ]:
# Write your code here


### 4.5. Aggregating data

But what about the most-featured Director in the top 1000 list?  Or the most common Production company?  For these kinds of operations we need to compute across aggregations of the dataset, and one way to think about this is in terms of a Split-Apply-Combine approach:[<sup>*</sup>](#fn)
- *Split* a dataset into relevant subsets;
- *Apply* a function to each subset;
- *Combine* all the pieces back together.

<img src="https://miro.medium.com/max/780/1*w2oGdXv5btEMxAkAsz8fbg.png">

<img src="https://swcarpentry.github.io/r-novice-gapminder/fig/12-plyr-fig1.png" alt="Split-Apply-Combine">

[Split-Apply-Combine](https://swcarpentry.github.io/r-novice-gapminder/fig/12-plyr-fig1.png) - Source: [Software Carpentry](https://software-carpentry.org/lessons/).

<hr>
<span id="fn">* A classic paper on the Split-Apply-Combine methodology is available here: [“The Split-Apply-Combine Strategy for Data Analysis”](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf)</span> 

In Pandas this can take the form of a `.groupby()` (split) operation followed by an `.aggregate()` (apply) function. Aggregates are like `apply()` that operate at the group level. Combining is done automatically for us by Pandas.

In [ ]:
df.groupby('Director')

In [ ]:
df.groupby('Director')[['Title']]

In [ ]:
df.groupby('Director')[['Title']].aggregate(lambda values: len(values))

In [ ]:
df.groupby('Director')[['Title']].aggregate(lambda values: len(values)).sort_values(['Title'], ascending=False)

In [ ]:
df.pivot_table(
    index=['Director', 'Production'],
    values=['Title'],
    aggfunc=len
).sort_values('Title', ascending=False).head(10)

In [ ]:
# df.loc[pd.isna(df.Genre), 'Genre'] = 'Comedy'


### 4.5.1. Activity

Show the top five years for movies categorized as "Comedy"

In [ ]:
# Write your code here
df[df.Genre.str.contains('Comedy')].groupby('Year')[['Title']].aggregate(len).sort_values(['Title'], ascending=False).head(5)

## 5. Visualization

Pandas also provides some utilities to create basic plots just by calling `plot()` on a `Series` or `DataFrame`. But first we need to tell Jupyter that we are going to plot some charts using the plotting library `matplotlib`.

In [ ]:
# enables inline plotting in Jupyter using matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df.groupby('Year')[['Year']].aggregate(len).plot()

Each time you call `plot()` an `AxesSubplot` object is returned, and Jupyter knows how to paint those. `AxesSubplot` objects are objects of the underlying `matplotlib` library for plotting in Python, and as such, lots of different options can be given to customize the aspect.

In [ ]:
ax = df.groupby('Year')[['Year']].aggregate(len).plot(
    kind="bar",
    figsize=(15, 5),
    title="# Movies per Year",
    legend=None
)
ax.set_ylabel("# Movies")
ax.set_xlabel("Year of Release")

In [ ]:
plt.style.available

In [ ]:
with plt.style.context('ggplot'):
    df.groupby('Year')[['Year']].aggregate(len).plot()

In [ ]:
with plt.xkcd():
    df.groupby('Year')[['Year']].aggregate(len).plot()
plt.rcdefaults()  # this is neede as XKCD style is a special case